# MQTT in a Nutshell

A self-containing notebook - no other notebook needs to be started

Illustrates the basic concepts of MQTT


**Discussion**: What is the difference between a database and a message broker?


Install paho

In [22]:
import paho.mqtt.client as mqtt #basic mqtt utilities

import time #for sleep

import random # random

import json #for json

from random import randint 

The library is not perfect: look at https://pypi.org/project/paho-mqtt/ - Known Issues

In [23]:
#contains all messages that are received for debugging purposes
all_messages=[]

IP of the broker

In [24]:
mqtt_broker_ip="127.0.0.1"

In [25]:
mqtt_broker_port=1883 #default mqtt broker port

**Publish and Subscribe**

Create some clients (need not to run on the same machine!). Each client can publish and subscribe.

In [26]:
client_ids=random.sample(range(1, 300), 3)
client_ids

[272, 129, 45]

In [27]:
mqtt_client_1=mqtt.Client(client_id="myClient-"+ str(client_ids[0]), clean_session=True, userdata="Client 1", protocol=mqtt.MQTTv311, transport="tcp")
#transport: alternative websockets, userdata: user defined data of any type that is passed as the userdata parameter to callbacks

In [28]:
mqtt_client_2=mqtt.Client(client_id="myClient2-"+ str(client_ids[1]), clean_session=True, userdata="Client 2", protocol=mqtt.MQTTv311, transport="tcp") 

In [29]:
mqtt_client_3=mqtt.Client(client_id="myClient3-"+ str(client_ids[2]), clean_session=True, userdata="Client 3", protocol=mqtt.MQTTv311, transport="tcp") 

In [30]:
mqtt_client_1.connect(mqtt_broker_ip, port=mqtt_broker_port, keepalive=60*4) 
#keepalive indicates when the broker is disconnected from the broker- PINGREQ/PINGRES are used to check connectivity
mqtt_client_2.connect(mqtt_broker_ip, port=mqtt_broker_port, keepalive=60*4)
mqtt_client_3.connect(mqtt_broker_ip, port=mqtt_broker_port, keepalive=60*4)

0

In [31]:
# # example: publish values in 1-second intervals
# while(True):
#     mqtt_client_1.publish("robotcell1/temperature", random.random()*100)  #publish random values
#     time.sleep(2) #wait to seconds

In [32]:
mqtt_client_1.publish("robotcell1/temperature", payload=3, qos=0) #qoc: quality of service, 0 means at most once, 1 means at least once, 2 means exactly once

**Back to Slides: to discuss QoS**

In [33]:
mqtt_client_1.publish("robotcell1/temperature", payload=4, qos=1)

In [34]:
#will not work
msg=mqtt_client_1.publish("robotcell1/temperature", payload=19, qos=2)

In [35]:
mqtt_client_1.loop_start()  #starts a loop in a thread
msg=mqtt_client_1.publish("robotcell1/temperature", payload=19, qos=2)

In [36]:
msg.is_published()

False

In [37]:
#mqtt_client_1.loop_stop()

Why is loop requiered here?

-> To handle the extra packets needed for high QoS

## Payload

MQTT makes no constraints regarding the payload - so JSON, XML as well as raw values are allowed!

In [38]:
mqtt_client_1.publish("robotcell1/temperature", payload=3, qos=0)

In [39]:
#create dict
payload ={ 
  "temperatur":3,
  "type":"robot front"
} 
      
# Serializing json  
json_object = json.dumps(payload, indent = 4) 

mqtt_client_1.publish("robotcell1/temperature", payload=json_object, qos=0)

## Subscribe

In [40]:
def on_message(client, userdata, msg):
    print("Client " + str(userdata) + " " + msg.topic + " " + str(msg.qos) + " " + str(msg.payload) + " retained: " + str(msg.retain))
    
    #append to list of all messages
    all_messages.append(msg)

In [41]:
mqtt_client_2.connect(mqtt_broker_ip, port=mqtt_broker_port, keepalive=60*4)
mqtt_client_2.loop_start()

### alternative to loop_start()
#import threading

#subscribe_thread=threading.Thread(target=mqtt_client_2.loop_forever)  # loop_forever is blocking!
#subscribe_thread.start()
mqtt_client_2.on_message=on_message #register callback

In [42]:
mqtt_client_2.subscribe("robotcell1/temperature", qos=0) #subscribe to topic

(0, 1)

In [43]:
#mqtt_client_2.loop_stop()

In [44]:
mqtt_client_3.connect(mqtt_broker_ip, port=mqtt_broker_port, keepalive=60*4)
mqtt_client_3.loop_start()
mqtt_client_3.on_message=on_message 
mqtt_client_3.subscribe("robotcell1/temperature", qos=0) #subscribe to topic


(0, 1)

In [45]:
#mqtt_client_3.publish("robotcell1/temperature",payload=5,qos=0)

In [46]:
#mqtt_client_3.loop_stop()

Now, parse the content as JSON

In [47]:
#def on_message_json(client, obj, msg):
#    print(msg.topic + " " + str(msg.qos) + " " + str(json.loads(msg.payload)))

In [48]:
#mqtt_client_2.on_message=on_message_json #register callback

In [49]:
# mqtt_client_2.loop_forever()

# In-Class Exercise

Assume the following structure in MQTT:


`robotcell1/temperature1=21`

`robotcell1/temperature2=26`


Simply create it in the broker by publishing to those topics.


Create a client, that subscribes to them and publishes the average to `robotcell1/avgtemperature`


# Wildcards (already discussed)

In [50]:
#mqtt_client_2.loop_stop()

mqtt_client_2.subscribe("robotcell1/#", qos=0) #subscribe to topic

#mqtt_client_2.loop_start()
#mqtt_client_2.loop_stop()

(0, 2)

In [51]:
mqtt_client_2.unsubscribe("robotcell1/#")
mqtt_client_2.subscribe("robotcell1/+", qos=0) #subscribe to topic
mqtt_client_2.loop_start()

3

In [52]:
mqtt_client_2.loop_stop()

In [53]:
mqtt_client_2.subscribe("robotcell1/+/int/+", qos=0) #subscribe to topic

(0, 5)

In [54]:
mqtt_client_2.loop_start()

# In-Class Exercise

Create three clients (for three system components) that represent system components pushing cylinders. Cylinder 2 gets active after Cylinder 1 is in position, Cylinder 3 gest active after Cylinder 2 is in position.

Each Cylinder is represented by a topic

**Start Topics**:
    
`systemcomponent1/cylinderOn=False`

`systemcomponent2/cylinderOn=False`

`systemcomponent3/cylinderOn=False`


* The second system components should subscribe to the `cylinderOn` topic of the first system component and set the value of topic `systemcomponent2/cylinderOn=True` if  `systemcomponent1/cylinderOn=True`
* The thrid system component should subscribe to the `cylinderOn` topic of the second system component and set the value of topic `systemcomponent3/cylinderOn=True` if  `systemcomponent2/cylinderOn=True`
* The first system component should subscribe to the `cylinderOn` topic of the third system component and set the value of topic `systemcomponent1/cylinderOn=False` if  `systemcomponent3/cylinderOn=True`.

In order to test the setup, simply publish `True` to `systemcomponent1/cylinderOn`: then the value of the first cylinder should become False, while the other values should become True.



### Back to the slides

# Retained vs Non-Retained

In [55]:
mqtt_client_1.publish("robotcell1/temperature",payload=2,qos=0,retain=True)  #retained is false in the usual message flow

Client Client 3 robotcell1/temperature 0 b'2' retained: 0
Client Client 3 robotcell1/temperature 0 b'75' retained: 0


Client Client 2 robotcell1/temperature 0 b'2' retained: 0
Client Client 2 robotcell1/temperature 0 b'75' retained: 0
Client Client 2 robotcell1/joints 0 b'{"joint0": 10, "joint1": 42, "joint2": 53, "joint3": 64, "joint4": 8, "joint5": 22}' retained: 0
Client Client 2 robotcell1/temperature 0 b'76' retained: 0
Client Client 2 robotcell1/temperature 0 b'39' retained: 0
Client Client 2 robotcell1/temperature 0 b'77' retained: 0
Client Client 2 robotcell1/temperature 0 b'99' retained: 0
Client Client 2 robotcell1/temperature 0 b'39' retained: 0
Client Client 2 robotcell1/temperature 0 b'77' retained: 0
Client Client 2 robotcell1/temperature 0 b'99' retained: 0
Client Client 2 robotcell1/temperature1 0 b'21' retained: 0
Client Client 2 robotcell1/temperature1 0 b'26' retained: 0
Client Client 2 robotcell1/temperature1 0 b'21' retained: 0
Client Client 2 robotcell1/temperature1 0 b'26' retained: 0
Client Client 2 robotcell1/temperature1 0 b'21' retained: 0
Client Client 2 robotcell1/temperat

In [56]:
mqtt_client_1.publish("robotcell1/temperature",payload=75,qos=0,retain=False)

In [57]:
mqtt_client_4=mqtt.Client(client_id="myClient4", clean_session=True, userdata="Client 4", protocol=mqtt.MQTTv311, transport="tcp") 

In [58]:
mqtt_client_4.on_message=on_message
mqtt_client_4.connect(mqtt_broker_ip, port=1883, keepalive=60*4) 
mqtt_client_4.loop_start()

What payload will a new subscriber get?

In [59]:
mqtt_client_4.subscribe("robotcell1/temperature", qos=0) 

(0, 1)

In [60]:
mqtt_client_4.loop_stop()

Client Client 4 robotcell1/temperature 0 b'2' retained: 1


In [61]:
mqtt_client_4.disconnect()

0

# Design

We can use differnt payloads in MQTT messages: from raw data values to complete JSON documents

Assume we have a robot with six axis and we want to publish the data to MQTT

In [62]:
robot_joints=[10,42,53,64,8,22]  #for each joint we have a value representing the current position (degree)

In [63]:
#Flatten-out the data
for idx,joint in enumerate(robot_joints):
    topic="robotcell1/joints/{0}".format(idx)
    print("...publish to " + topic )
    mqtt_client_1.publish(topic,payload=joint,qos=0)

...publish to robotcell1/joints/0
...publish to robotcell1/joints/1
...publish to robotcell1/joints/2
...publish to robotcell1/joints/3
...publish to robotcell1/joints/4
...publish to robotcell1/joints/5


In [64]:
#Compact format...

joints={}
for idx,joint in enumerate(robot_joints):
    joints["joint"+str(idx)]=joint

In [65]:
mqtt_client_1.publish("robotcell1/joints",payload=json.dumps(joints),qos=0)

**Discussion** 

What are the drawbacks/benefits of the two approaches:

* Data-Traffic when a joint is updated
* Effort for publishing data
* Serializing data to that structure and the data back?

# Clean Session

Create a client as before - with `clean_session=True`

In [66]:
mqtt_client_5=mqtt.Client(client_id="myClient5", clean_session=True, userdata="client 5", protocol=mqtt.MQTTv311, transport="tcp")  # clean session=True - same as on the top
time.sleep(1)

In [67]:
mqtt_client_5.on_message=on_message 
mqtt_client_5.connect(mqtt_broker_ip, port=1883, keepalive=60*4)
mqtt_client_5.loop_start()
time.sleep(1)

Create another client - with `clean_session=False`

In [68]:
mqtt_client_6=mqtt.Client(client_id="myClient6", clean_session=False, userdata="client 6", protocol=mqtt.MQTTv311, transport="tcp")  # clean session=False

In [69]:
mqtt_client_6.on_message=on_message
mqtt_client_6.connect(mqtt_broker_ip, port=1883, keepalive=60*4) 
mqtt_client_6.loop_start()
time.sleep(1)

Subscribe to the temperature topic

In [70]:
mqtt_client_6.subscribe("robotcell1/temperature",qos=0)
time.sleep(2)

Client client 6 robotcell1/temperature 0 b'2' retained: 1


In [71]:
mqtt_client_5.subscribe(topic="robotcell1/temperature",qos=0)
time.sleep(1)

Client client 5 robotcell1/temperature 0 b'2' retained: 1


So far, both get the same values

**simulate a disconnection of the two clients (e.g. a connection drop down...)**

In [72]:
mqtt_client_6.disconnect()
mqtt_client_6.loop_stop()
time.sleep(1)

In [73]:
mqtt_client_5.disconnect()
mqtt_client_5.loop_stop()
time.sleep(1)

In [74]:
#now new data is published which was relevant for the clients....
mqtt_client_1.publish("robotcell1/temperature",payload=76)  # NON RETAINED!!! QoS is 0 by default!

Client Client 3 robotcell1/temperature 0 b'76' retained: 0
Client Client 3 robotcell1/temperature 0 b'39' retained: 0
Client Client 3 robotcell1/temperature 0 b'77' retained: 0
Client Client 3 robotcell1/temperature 0 b'99' retained: 0
Client Client 3 robotcell1/temperature 0 b'39' retained: 0
Client Client 3 robotcell1/temperature 0 b'77' retained: 0
Client Client 3 robotcell1/temperature 0 b'99' retained: 0


**simulate reconnection of the clients**

In [75]:
mqtt_client_6.reconnect()  #subscription is remembered
mqtt_client_6.loop_start()
time.sleep(1)

In [76]:
mqtt_client_5.reconnect()   #subscription is NOT remembered 
mqtt_client_5.loop_start()
time.sleep(1)

In [77]:
#retained values is returned!
mqtt_client_5.subscribe(topic="robotcell1/temperature",qos=0)

(0, 2)

Client client 5 robotcell1/temperature 0 b'2' retained: 1


The message sent in the meantime is lost!

**Disconnect Again**

In [78]:
mqtt_client_6.disconnect()
mqtt_client_6.loop_stop()
time.sleep(1)

In [79]:
mqtt_client_5.disconnect()
mqtt_client_5.loop_stop()
time.sleep(1)

In [80]:
#publish with QoS 1

In [81]:
mqtt_client_1.publish("robotcell1/temperature",payload=39,qos=1)
mqtt_client_1.publish("robotcell1/temperature",payload=77,qos=1)
mqtt_client_1.publish("robotcell1/temperature",payload=99,qos=1)

In [82]:
all_messages

**Re-Connect Again**

In [83]:
mqtt_client_6.connect(mqtt_broker_ip, port=1883, keepalive=60*4)   
mqtt_client_6.loop_start()

In [84]:
mqtt_client_5.connect(mqtt_broker_ip, port=1883, keepalive=60*4)   
mqtt_client_5.loop_start()

Also here the message sent in between is lost! So lets move up the QoS of the subscription to 1

In [85]:
mqtt_client_6.subscribe("robotcell1/temperature",qos=1)

(0, 2)

Client client 6 robotcell1/temperature 0 b'2' retained: 1


In [86]:
mqtt_client_5.subscribe("robotcell1/temperature",qos=1)

(0, 3)

Client client 5 robotcell1/temperature 0 b'2' retained: 1


**Disconnect**

In [87]:
mqtt_client_6.disconnect()
mqtt_client_6.loop_stop()
time.sleep(1)

In [88]:
mqtt_client_5.disconnect()
mqtt_client_5.loop_stop()
time.sleep(1)

**Sent QoS Messages in Between**

In [89]:
mqtt_client_1.publish("robotcell1/temperature",payload=39,qos=1)
mqtt_client_1.publish("robotcell1/temperature",payload=77,qos=1)
mqtt_client_1.publish("robotcell1/temperature",payload=99,qos=1)

`Client 6` now receives all the **NON-RETAINED** values!

In [90]:
mqtt_client_6.reconnect()
mqtt_client_6.loop_start()

Client client 6 robotcell1/temperature 1 b'39' retained: 0
Client client 6 robotcell1/temperature 1 b'77' retained: 0
Client client 6 robotcell1/temperature 1 b'99' retained: 0


In [ ]:
mqtt_client_5.reconnect()
mqtt_client_5.loop_start()
mqtt_client_5.subscribe("robotcell1/temperature",qos=1)

(0, 4)

Client client 5 robotcell1/temperature 0 b'2' retained: 1
